In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import torch
import pickle
import sys
sys.path.append('../acd/acd')

In [18]:
import os
vector_cache = os.path.join(os.getcwd(), '.vector_cache/input_vectors.pt')
word_vectors ='glove.6B.300d'
batch_size=  50


['model1.pt', 'model2.pt', 'model3.pt']

In [30]:

sys.path.append('../acd/visualization')
sys.path.append('../acd/acd/util')
sys.path.append('../acd/acd/scores')
sys.path.append('../acd/acd/agglomeration')
import viz_1d as viz
import tiling_1d as tiling
import agg_1d as agg
import cd
import score_funcs

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# form class to hold data
class B:
    text = torch.zeros(1).to(device)
sys.path.append('../acd')
sys.path.append('../acd/dsets/sst')
from dsets.sst import dset
from dsets.sst.model import LSTMSentiment

# check out how two models differ
import torch.optim as O
import torch.nn as nn

from torchtext import data
from torchtext import datasets

In [31]:

inputs = data.Field(lower= True)
answers = data.Field(sequential=False, unk_token=None)

train, dev, test = datasets.SST.splits(inputs, answers, fine_grained=False, train_subtrees=True,
                                       filter_pred=lambda ex: ex.label != 'neutral')

inputs.build_vocab(train, dev, test)
if word_vectors:
    if os.path.isfile(vector_cache):
        inputs.vocab.vectors = torch.load(vector_cache)
    else:
        inputs.vocab.load_vectors(word_vectors)
        makedirs(os.path.dirname(vector_cache))
        torch.save(inputs.vocab.vectors,vector_cache)
answers.build_vocab(train)

train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=batch_size, device=torch.device(0))

In [34]:

model = torch.load('models/model3.pt', map_location=torch.device(0))
criterion = nn.CrossEntropyLoss()
n_dev_correct, dev_loss = 0, 0
for dev_batch_idx, dev_batch in enumerate(dev_iter):


    answer1 = model(dev_batch)
    n_dev_correct += ((torch.max(answer1, 1)[1].view(dev_batch.label.size()).data == dev_batch.label.data)).sum()
    dev_loss = criterion(answer1, dev_batch.label)
dev_acc = 100. * n_dev_correct / len(dev)
print(dev_acc.item())

84
